Agent's are initialized with the following variable parameters:

$P_u\ =\ \text{payoff for unlocking door}$

$\gamma_u\ =\ \text{expected risk of being robbed}$

$L_u\ =\ \text{loss from being robbed}$

$P_t\ =\ \text{payoff to successfull theft}$

$\gamma_t\ =\ \text{expected probabilty of successfull theft}$

$L_t\ =\ \text{loss from any attempted theft}$


Agent's utility functions for locking the door and attmepting theft are defined as:

$$
U_{door} = P_u - \gamma_uL_u
$$
$$
U_{theft} = \gamma_tP_t - L_t
$$

where all paramaters are > 0. 

Agent's only take an action if the actions has a positive expected utility, so the agent will lock her door when payoff to unlocking the door outweights the expected loss
$$
P_u > \gamma_uL_u 
$$
and the agent will attempt a theft when the expected payoff is greater than the consequences
$$
\gamma_tP_t > - L_t
$$

Crucially, both equations depend on the agent's beliefs about risk, captured in $\gamma_u, \ \gamma_t$, which will be the only parameters that change in the sumulation.



In [81]:
class Agent(object):
    
    def __init__(self, parameters):
        for k,v in parameters.items(): #loop through all keyword arguments
            # assert(k in self.__class__.__allowed) #make sure all the obejcts passed are compatible with base class, if applicable
            setattr(self,k,v) #assign the kwargs as object attributes
        self.lock = not bool(self.unlockPayoff > self.unlockRisk * self.unlockPenalty) #decicde whether to lock door(lock unless the payoff to unlock is higher than the square of the risk of theft)
        self.theft = bool(self.theftPenalty < (1-self.theftRisk) * self.theftPayoff) #decide whether to attempt a robbery (attempt if the payoff to robbing is higher than the probability of failure)
    
    
    
    

In [115]:
testparams = {'unlockPayoff':.5, 'unlockRisk':.2, 'unlockPenalty':.1, 'theftPayoff':1.01, 'theftRisk':.001,'theftPenalty':1,}
test = Agent(testparams)

Agents slowly learn about the current rate (lagged true values)  
